In [150]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [151]:
# if using mac
df = pd.read_excel('/Users/zhanghantang/PycharmProjects/sucide_project/BIOM40forUSC.xlsx')
data = df[df['SI'].notnull()]
y = data['SI']
x = data.loc[:, 'GIMAP1Biom1552316_a_at':'CFIS']

In [71]:
# windows
df = pd.read_excel('E:\PHD\sucide_project\BIOM40forUSC.xlsx')
data = df[df['SI'].notnull()]
y = data['SI']
x = data.loc[:, 'GIMAP1Biom1552316_a_at':'CFIS']

FileNotFoundError: [Errno 2] No such file or directory: 'E:\\PHD\\sucide_project\\BIOM40forUSC.xlsx'

In [152]:
# column 'CFI-S.PheneVisit' data type is string, i am not clear its internal meaning and how to convert to float data type
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
x = x.drop(labels='CFI-S.PheneVisit', axis=1)
# drop these column directly
x = data.loc[:, 'GIMAP1Biom1552316_a_at':'RAB3GAP2Biom240234_at']
new_y = []
for i in y:
    if i == 0 or i ==1:
        new_y.append(0)
    else:
        new_y.append(1)
y = np.array(new_y)
X = StandardScaler().fit_transform(x)
X_train, X_test, y_train, y_test = train_test_split(X, new_y, test_size=0.2, random_state=10)
print(len(X_train), len(X_test))

# todo unified randome state
unified_random_state = 32

603 151


In [ ]:
'''
-------------------------
the content below is about logistic regression content
-------------------------
'''

In [112]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression as LR

# test parameter individually

# Create the K-Fold object
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Define your model
model = LR(solver='saga', penalty='elasticnet', C=100, max_iter=3000, l1_ratio=0.5)

# Use the cross_val_score function to obtain the scores for each fold
scores = cross_val_score(model, X, new_y, cv=kf, scoring='f1')


# Print the scores for each fold and the mean score
print("Scores: ", scores)
print("Mean Score: ", scores.mean())

'''
Scores:  [0.24390244 0.11764706 0.35897436 0.25531915 0.18181818]
Mean Score:  0.23153223751532614
'''

Scores:  [0.24390244 0.11764706 0.35897436 0.25531915 0.18181818]
Mean Score:  0.23153223751532614


'\nScores:  [0.24390244 0.11764706 0.35897436 0.25531915 0.18181818]\nMean Score:  0.23153223751532614\n'

In [108]:
# tune parameter
from sklearn.model_selection import GridSearchCV
model = LR(solver='saga')
# 不同的solver可能只支持不同的有限的penalty
param_grid = {'penalty':['l1', 'l2'], 'C':[0.1, 1, 10, 100], 'max_iter':[3000, 3500]}
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1', return_train_score=True)
grid_search.fit(X, new_y)
print("Best penalty: ", grid_search.best_params_['penalty'])
print("Best C: ", grid_search.best_params_['C'])
print("Best score: ", grid_search.best_score_)

'''
Best penalty:  l1
Best C:  100
Best score:  0.19299478970021897

'''

Best penalty:  l1
Best C:  100
Best score:  0.19299478970021897


In [113]:
best_lr_model = LR(C=100, penalty='l1', solver='saga', max_iter=3000)
best_lr_model.fit(X_train, y_train)
y_pred = best_lr_model.predict(X_test)
print("Score with L1 penalty: accuracy:{} precision:{} recall:{} f1:{} roc_auc:{}".format(accuracy_score(y_test, y_pred), precision_score(y_test, y_pred), recall_score(y_test, y_pred), f1_score(y_test, y_pred), roc_auc_score(y_test, y_pred)))
'''
Score with L1 penalty: accuracy:0.7831325301204819 precision:0.4074074074074074 recall:0.22448979591836735 f1:0.2894736842105263 roc_auc:0.5722448979591838
'''

Score with L1 penalty: accuracy:0.7831325301204819 precision:0.4074074074074074 recall:0.22448979591836735 f1:0.2894736842105263 roc_auc:0.5722448979591838


In [ ]:
'''
-------------------------
the content below is about SVM
-------------------------
'''

In [188]:
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

model = SVC(C=100, kernel='rbf', max_iter=10000)
scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
scores = cross_validate(model, X, y, cv=5, scoring=scoring)
# y_pred = model.predict(X)
print("Scores: accuracy is {}, mean accuary is {}".format(scores['test_accuracy'], scores['test_accuracy'].mean()))
print("Scores: precision is {}, mean accuary is {}".format(scores['test_precision'], scores['test_precision'].mean()))
print("Scores: recall is {}, mean accuary is {}".format(scores['test_recall'], scores['test_recall'].mean()))
print("Scores: f1 is {}, mean accuary is {}".format(scores['test_f1'], scores['test_f1'].mean()))
print("Scores: roc_auc is {}, mean accuary is {}".format(scores['test_roc_auc'], scores['test_roc_auc'].mean()))

'''
Scores: accuracy is [0.7615894  0.74834437 0.74172185 0.7615894  0.58      ], mean accuary is 0.7186490066225166
Scores: precision is [0.29411765 0.15384615 0.33333333 0.33333333 0.26027397], mean accuary is 0.27498088803487675
Scores: recall is [0.17241379 0.06896552 0.34482759 0.28571429 0.67857143], mean accuary is 0.31009852216748773
Scores: f1 is [0.2173913  0.0952381  0.33898305 0.30769231 0.37623762], mean accuary is 0.26710847637761254
Scores: roc_auc is [0.56783493 0.60655738 0.65008479 0.71544715 0.69818501], mean accuary is 0.6476218543781174

'''

Scores: accuracy is [0.7615894  0.74834437 0.74172185 0.7615894  0.58      ], mean accuary is 0.7186490066225166
Scores: precision is [0.29411765 0.15384615 0.33333333 0.33333333 0.26027397], mean accuary is 0.27498088803487675
Scores: recall is [0.17241379 0.06896552 0.34482759 0.28571429 0.67857143], mean accuary is 0.31009852216748773
Scores: f1 is [0.2173913  0.0952381  0.33898305 0.30769231 0.37623762], mean accuary is 0.26710847637761254
Scores: roc_auc is [0.56783493 0.60655738 0.65008479 0.71544715 0.69818501], mean accuary is 0.6476218543781174


In [145]:
from sklearn.model_selection import GridSearchCV
model = SVC(kernel='rbf', max_iter=10000)
param_grid = {'C':[1, 10, 100], }
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1', return_train_score=True)
grid_search.fit(X, y)

# print("Best penalty: ", grid_search.best_params_['kernel'])
print("Best C: ", grid_search.best_params_['C'])
print("Best score: ", grid_search.best_score_)

'''
Best penalty:  rbf
Best C:  100

Scores:  [0.36065574 0.44067797 0.45901639 0.31111111 0.40740741]
Mean Score:  0.39577372315355086
Scores:  [0.74274194 0.71833931 0.72649573 0.70439633 0.69226137]
Mean Score:  0.7168469323092028

'''

Best C:  100
Best score:  0.26710847637761254


'\nBest penalty:  rbf\nBest C:  10\nScores:  [0.41666667 0.3902439  0.45454545 0.30769231 0.38596491]\nMean Score:  0.391022648724831\n'

In [ ]:
'''
-------------------------
the content below is about random forest
-------------------------
'''

In [190]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
param_grid = {'n_estimators':[10, 100, 200], 'max_depth':[4, 5, 6, 7, 8, 9], 'min_samples_split': [2, 3, 4],
              'min_samples_leaf': [1, 2, 3]}


rf_model = RandomForestClassifier(random_state=32)
grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='f1')
grid_search.fit(X, y)

print(grid_search.cv_results_)
print(grid_search.best_params_)

{'mean_fit_time': array([0.01734514, 0.16068573, 0.28216157, 0.01430664, 0.14128737,
       0.28185511, 0.01490216, 0.14494061, 0.31622529, 0.02696857,
       0.15756593, 0.31592894, 0.01781864, 0.17361803, 0.32370877,
       0.01726084, 0.15608826, 0.30447612, 0.01571198, 0.1448916 ,
       0.30634384, 0.01821537, 0.17364182, 0.34628668, 0.01830478,
       0.17421179, 0.34690442, 0.01754675, 0.18497791, 0.3400641 ,
       0.01735907, 0.17546306, 0.33989615, 0.01592422, 0.16006503,
       0.31505375, 0.01705775, 0.16530571, 0.33389678, 0.01631627,
       0.16559253, 0.35043759, 0.01865635, 0.16335206, 0.32897782,
       0.0180346 , 0.16445827, 0.32268572, 0.01877532, 0.16420498,
       0.32088132, 0.01587014, 0.15290375, 0.33746881, 0.01755424,
       0.16485071, 0.33535342, 0.01709714, 0.16423483, 0.32784171,
       0.01677923, 0.16552539, 0.32664032, 0.01699338, 0.16298189,
       0.32894125, 0.0170351 , 0.16457834, 0.33428617, 0.01688228,
       0.16547685, 0.3286665 , 0.01673207, 0

In [202]:
from sklearn.model_selection import cross_validate

rf_model = RandomForestClassifier(n_estimators=10, max_depth=9, min_samples_leaf=1, min_samples_split=4, random_state=32)

scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']

scores = cross_validate(rf_model, X, y, cv=5, scoring=scoring)
print("Scores: accuracy is {}, mean accuary is {}".format(scores['test_accuracy'], scores['test_accuracy'].mean()))
print("Scores: precision is {}, mean precision is {}".format(scores['test_precision'], scores['test_precision'].mean()))
print("Scores: recall is {}, mean recall is {}".format(scores['test_recall'], scores['test_recall'].mean()))
print("Scores: f1 is {}, mean f1 is {}".format(scores['test_f1'], scores['test_f1'].mean()))
print("Scores: roc_auc is {}, mean roc_auc is {}".format(scores['test_roc_auc'], scores['test_roc_auc'].mean()))

# print("Score with L1 penalty: accuracy:{} precision:{} recall:{} f1:{} roc_auc:{}".format(accuracy_score(y_test, y_pred), precision_score(y_test, y_pred), recall_score(y_test, y_pred), f1_score(y_test, y_pred), roc_auc_score(y_test, y_pred)))
'''
Scores:  [0.37209302 0.31578947 0.34042553 0.1875     0.37837838]
Mean Score:  0.31883728144665924
Scores:  [0.77956989 0.72371565 0.79839115 0.77788714 0.75127974]
Mean Score:  0.7661687138135271

'''

Scores: accuracy is [0.79470199 0.80794702 0.81456954 0.81456954 0.76666667], mean accuary is 0.799690949227373
Scores: precision is [0.         0.5        0.6        0.5        0.37037037], mean precision is 0.3940740740740741
Scores: recall is [0.         0.03448276 0.10344828 0.03571429 0.35714286], mean recall is 0.1061576354679803
Scores: f1 is [0.         0.06451613 0.17647059 0.06666667 0.36363636], mean f1 is 0.13425794951411651
Scores: roc_auc is [0.49166196 0.40276993 0.65220464 0.60307782 0.70418618], mean roc_auc is 0.5707801033937822


'\nScores:  [0.37209302 0.31578947 0.34042553 0.1875     0.37837838]\nMean Score:  0.31883728144665924\nScores:  [0.77956989 0.72371565 0.79839115 0.77788714 0.75127974]\nMean Score:  0.7661687138135271\n\n'